Firstly, we import necessarcy packages, eg. MatCalc, MatGL and Pymatgen.

In [ ]:
from __future__ import annotations

import json
import random
import warnings

import matgl
import numpy as np
import pandas as pd
from matcalc.utils import PESCalculator
from matcalc.elasticity import ElasticityCalc
from matcalc.phonon import PhononCalc
from pymatgen.core import Structure

# To suppress warnings for clearer output
warnings.simplefilter("ignore")

In [ ]:
from scipy import constants

eVA3ToGPa = constants.e / (constants.angstrom) ** 3 / constants.giga  # noqa:N816

Next, I will show you how to run MatCalc-Bench. Here, TensorNet-MatPES-PBE-v2025.1-PES and M3GNet-MatPES-PBE-v2025.1-PES are used to perform elastic modulus calculations. The approach is similar for other models and other benchmarks. For demonstration purposes only, we will sample 100 structures from the entire test dataset.

In [ ]:
rows = []
structures = []
with open("MatCalc-Bench_data/elastic.json") as f:
    entries = json.load(f)

random.seed(2025)
entries = random.sample(entries, 100)

# We will first create a DataFrame from the required components from the raw data.
# We also create the list of structures in the order of the entries.
for entry in entries:
    rows.append({
        "mp_id": entry["mp_id"],
        "formula": entry["formula"],
        "K_DFT": entry["bulk_modulus_vrh"],
        "G_DFT": entry["shear_modulus_vrh"],
    })
    structures.append(Structure.from_dict(entry["structure"]))

results = pd.DataFrame(rows)

In [ ]:
for model_name in [
    "M3GNet-MatPES-PBE-v2025.1-PES",
    "TensorNet-MatPES-PBE-v2025.1-PES",
]:
    calculator = PESCalculator.load_universal(model_name)

    elastic_calc = ElasticityCalc(calculator, fmax=0.05, relax_structure=True)

    # We use trivial parallel processing in joblib to speed up the computations.
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        properties = list(elastic_calc.calc_many(structures, n_jobs=-1))

    results[f"K_{model_name}"] = [d["bulk_modulus_vrh"] * eVA3ToGPa for d in properties]
    results[f"G_{model_name}"] = [d["shear_modulus_vrh"] * eVA3ToGPa for d in properties]
    results[f"AE K {model_name}"] = np.abs(results[f"K_{model_name}"] - results["K_DFT"])
    results[f"AE G {model_name}"] = np.abs(results[f"G_{model_name}"] - results["G_DFT"])

print(results)

/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/dgl/core.py:82: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/ase/filters.py:600: RuntimeWarning: logm result may be inaccurate, approximate err = 2.222647387488954e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor
/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/ase/filters.py:600: RuntimeWarning: logm result may be inaccurate, approximate err = 5.140338835627472e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor
/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/ase/filters.py:600: RuntimeWarning: logm result may be inaccurate, approximate err = 5.080568772648669e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor
/Users/shyue/miniconda3/envs/mavrl/lib/python3.11/site-packages/ase/filters.py:600: RuntimeWarning: logm res

      mp_id formula    K_DFT    G_DFT  K_M3GNet-MatPES-PBE-v2025.1-PES  \
0      1087     SrS   48.468   32.300                        41.469246   
1      7951   ThSe2   77.932   26.736                        32.900799   
2      9813     WS2   28.713   22.737                        99.015250   
3       745   ThMg2   50.194   18.428                        30.766925   
4      2816   Nd2Tl   45.139   17.053                        23.120002   
..      ...     ...      ...      ...                              ...   
95  1018148   ScAu2  117.569   46.472                       111.737436   
96  1001833    InBi   35.088   14.188                        77.275547   
97     2294   Al3Ir  150.642   34.510                       103.419816   
98     1597     UO2  206.152   92.078                       153.144879   
99   867816   TaRu3  276.055  139.600                       276.133164   

    G_M3GNet-MatPES-PBE-v2025.1-PES  AE K M3GNet-MatPES-PBE-v2025.1-PES  \
0                         26.790244 

In [ ]:
# To dump the results to a csv file, uncomment the code below.
# results.to_csv("MatCalc-Benchmark-elasticity.csv")

You can certainly perform the full benchmark by slightly modifying the codes provided below, but please be aware that the whole process is considerably time consuming. Therefore, I recommend submitting separate jobs to your cluster for every material—or every few materials, depending on your resource availability and workflow preferences.